In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'o3': 'atmpro'}
band = [7]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.6, 1.75), (1.55, 1.66, 1.7, 1.75, 1.8)]
  wgt = [(0.35, 0.6), (0.5, 0.55, 0.7, 0.9, 1.0)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.028373                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.016447
0.000750      2.0 -0.029881                  2.0 -0.016911
0.001052      3.0 -0.029819                  3.0 -0.015836
0.001476      4.0 -0.025145                  4.0 -0.011372
0.002070      5.0 -0.015295                  5.0 -0.003148
0.002904      6.0  0.000422                  6.0  0.009392
0.004074      7.0  0.022769                  7.0  0.026826
0.005714      8.0  0.052561                  8.0  0.049773
0.008015      9.0  0.090603                  9.0  0.078927
0.011243     10.0  0.137577                 10.0  0.115045
0.015771     11.0  0.194559                 11.0  0.159080
0.022122     12.0  0.262208                 12.0  0.211727
0.031031     13.0  0.341011                 13.0  0.273646
0.043528     14.0  0.460131                 14.0  0.369240
0.061057     15.0  0.637573                 15.0  0.515379
0.085645     16.0  0.842384                 16.0  0.690537
0.120136     17.0  1.059506                 17.0  0.886245
0.168516     18.0  1.281983                 18.0  1.100484
0.236378     19.0  1.502513                 19.0  1.329222
0.331549     20.0  1.717542                 20.0  1.566826
0.465100     21.0  1.919296                 21.0  1.794699
0.652400     22.0  1.966466                 22.0  1.897834
0.915100     23.0  1.657891                 23.0  1.670754
1.283650     24.0  1.248122                 24.0  1.280622
1.800600     25.0  0.925312                 25.0  0.935006
2.525700     26.0  0.647718                 26.0  0.650625
3.542800     27.0  0.425261                 27.0  0.435645
4.969550     28.0  0.284295                 28.0  0.299581
6.970850     29.0  0.210487                 29.0  0.231518
9.778100     30.0  0.157738                 30.0  0.178699
13.715850    31.0  0.112075                 31.0  0.128779
19.239350    32.0  0.065230                 32.0  0.076335
26.987250    33.0  0.036084                 33.0  0.042042
37.855300    34.0  0.022900                 34.0  0.024278
53.100050    35.0 -0.000917                 35.0 -0.002527
73.887500    36.0 -0.031218                 36.0 -0.036394
97.662500    37.0 -0.056988                 37.0 -0.067762
121.437500   38.0 -0.075784                 38.0 -0.091427
145.212500   39.0 -0.090571                 39.0 -0.109051
168.987500   40.0 -0.103127                 40.0 -0.122099
192.762500   41.0 -0.102940                 41.0 -0.117844
216.537500   42.0 -0.092992                 42.0 -0.101815
240.312500   43.0 -0.082576                 43.0 -0.086074
264.087500   44.0 -0.069529                 44.0 -0.068959
287.862500   45.0 -0.055781                 45.0 -0.052845
311.637500   46.0 -0.040812                 46.0 -0.037359
335.412500   47.0 -0.027997                 47.0 -0.025068
359.187500   48.0 -0.020220                 48.0 -0.017777
382.962500   49.0 -0.014663                 49.0 -0.012622
406.737500   50.0 -0.010081                 50.0 -0.008453
430.512500   51.0 -0.006344                 51.0 -0.005127
454.287500   52.0 -0.003649                 52.0 -0.002760
478.062500   53.0 -0.001654                 53.0 -0.000996
501.837500   54.0  0.000131                 54.0  0.000580
525.612500   55.0  0.001774                 55.0  0.002021
549.387500   56.0  0.003295                 56.0  0.003353
573.162500   57.0  0.004693                 57.0  0.004576
596.937500   58.0  0.005975                 58.0  0.005700
620.712500   59.0  0.007152                 59.0  0.006738
644.487500   60.0  0.008211                 60.0  0.007673
668.262500   61.0  0.009138                 61.0  0.008492
692.037500   62.0  0.009965                 62.0  0.009231
715.812500   63.0  0.010704                 63.0  0.009898
739.587500   64.0  0.011333                 64.0  0.010471
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -10.989699  0.000000 -10.989699        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -10.701487  0.000002   
6.244000e-04 2     -10.989703  0.000002 -10.989701 -10.701490  0.000004   
8.759000e-04 3     -10.989705  0.000003 -10.989702 -10.701491  0.000004   
1.228600e-03 4     -10.989708  0.000005 -10.989703 -10.701493  0.000006   
1.723400e-03 5     -10.989714  0.000010 -10.989704 -10.701497  0.000009   
2.417400e-03 6     -10.989722  0.000017 -10.989706 -10.701502  0.000014   
3.390900e-03 7     -10.989735  0.000030 -10.989706 -10.701510  0.000023   
4.756500e-03 8     -10.989754  0.000052 -10.989702 -10.701522  0.000039   
6.672000e-03 9     -10.989781  0.000091 -10.989690 -10.701539  0.000067   
9.358900e-03 10    -10.989818  0.000157 -10.989661 -10.701561  0.000115   
1.312780e-02 11    -10.989866  0.000266 -10.989600 -10.701590  0.000195   
1.841450e-02 12    -10.989927  0.000449 -10.989478 -10.701624  0.000329   
2.583020e-02 13    -10.989997  0.000750 -10.989248 -10.701659  0.000550   
3.623230e-02 14    -10.990068  0.001241 -10.988827 -10.701686  0.000914   
5.082340e-02 15    -10.990128  0.002096 -10.988032 -10.701688  0.001555   
7.129060e-02 16    -10.990156  0.003670 -10.986486 -10.701632  0.002747   
1.000000e-01 17    -10.990083  0.006461 -10.983621 -10.701438  0.004902   
1.402710e-01 18    -10.989752  0.011184 -10.978567 -10.700944  0.008636   
1.967600e-01 19    -10.988852  0.018863 -10.969989 -10.699841  0.014898   
2.759970e-01 20    -10.986785  0.030899 -10.955886 -10.697553  0.025088   
3.871000e-01 21    -10.982378  0.049097 -10.933281 -10.692999  0.041157   
5.431000e-01 22    -10.973343  0.075529 -10.897814 -10.684219  0.065545   
7.617000e-01 23    -10.957889  0.110996 -10.846893 -10.669612  0.100088   
1.068500e+00 24    -10.937587  0.150946 -10.786641 -10.650289  0.141490   
1.498800e+00 25    -10.913845  0.190823 -10.723022 -10.627332  0.183816   
2.102400e+00 26    -10.886872  0.230011 -10.656861 -10.600826  0.224170   
2.949000e+00 27    -10.859594  0.267690 -10.591904 -10.573711  0.262310   
4.136600e+00 28    -10.837164  0.305086 -10.532079 -10.551002  0.300894   
5.802500e+00 29    -10.823226  0.347250 -10.475976 -10.536261  0.345278   
8.139200e+00 30    -10.817017  0.399303 -10.417714 -10.528830  0.401938   
1.141700e+01 31    -10.819420  0.462952 -10.356468 -10.529741  0.472240   
1.601470e+01 32    -10.838048  0.542620 -10.295428 -10.546775  0.559419   
2.246400e+01 33    -10.891591  0.645996 -10.245595 -10.598814  0.669780   
3.151050e+01 34    -10.998317  0.791391 -10.206927 -10.705219  0.821244   
4.420010e+01 35    -11.165171  0.992667 -10.172504 -10.874864  1.027387   
6.200000e+01 36    -11.411265  1.236828 -10.174437 -11.132288  1.279483   
8.577500e+01 37    -11.755827  1.493470 -10.262356 -11.506370  1.551056   
1.095500e+02 38    -12.101077  1.678224 -10.422853 -11.896685  1.750511   
1.333250e+02 39    -12.439066  1.802781 -10.636285 -12.291596  1.887908   
1.571000e+02 40    -12.781920  1.890557 -10.891363 -12.697657  1.986815   
1.808750e+02 41    -13.139980  1.958180 -11.181800 -13.118844  2.064096   
2.046500e+02 42    -13.479483  2.007770 -11.471712 -13.508120  2.121452   
2.284250e+02 43    -13.776851  2.043243 -11.733608 -13.836347  2.162905   
2.522000e+02 44    -14.035847  2.069678 -11.966169 -14.109932  2.194053   
2.759750e+02 45    -14.251189  2.089203 -12.161986 -14.327307  2.217198   
2.997500e+02 46    -14.423198  2.104116 -12.319082 -14.493700  2.234746   
3.235250e+02 47    -14.552269  2.118248 -12.434021 -14.614418  2.250237   
3.473000e+02 48    -14.647075  2.134206 -12.512869 -14.701111  2.266323   
3.710750e+02 49    -14.722635  2.152819 -12.569816 -14.769101  2.284244   
3.948500e+02 50    -14.784491  2.173380 -12.611110 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')